In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import Concatenate
import pandas as pd


In [2]:
behaviors = pd.read_csv("../../../data/mind_small_train/behaviors_processed_small.csv")

In [3]:
behav = behaviors.to_numpy(copy=True)

In [4]:
behaviors['history'] = behaviors.history.str.split(' ')

In [5]:
behaviors.head()

,impression_id,user_id,time,history,impressions,length_history
0,1,U13740,11/11/2019 9:05:58 AM,"[N55189, N42782, N34694, N45794, N18445, N6330...",N55689-1 N35729-0,9
1,2,U91836,11/12/2019 6:11:30 PM,"[N31739, N6072, N63045, N23979, N35656, N43353...",N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,82
2,3,U73700,11/14/2019 7:01:48 AM,"[N10732, N25792, N7563, N21087, N41087, N5445,...",N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,16
3,4,U34670,11/11/2019 5:28:05 AM,"[N45729, N2203, N871, N53880, N41375, N43142, ...",N35729-0 N33632-0 N49685-1 N27581-0,10
4,6,U19739,11/11/2019 6:52:13 PM,"[N39074, N14343, N32607, N32320, N22007, N442,...",N21119-1 N53696-0 N33619-1 N25722-0 N2869-0,36


In [6]:
behav.shape

(136120, 6)

In [7]:
articles = []
for row in behav:
    for article in row[3].split(' '):
        articles.append(article)
        
    for article in row[4].split(' '):
        articles.append(article[:-2])

    
    

In [8]:
unique_articels = set(articles)

In [9]:
num_articles = len(unique_articels)

In [10]:
article_idx = {u:i for i, u in enumerate(unique_articels)}

In [11]:
behaviors['history_int'] = behaviors.history

In [12]:
behaviors['history_int'] = behaviors['history_int'].apply(lambda x: [article_idx[i] for i in x[-5:]])

In [13]:
behaviors.history_int[:3]

0    [19662, 4343, 29194, 46913, 26811]
1    [33409, 8855, 15451, 10733, 25150]
2      [30923, 28544, 7090, 17478, 680]
Name: history_int, dtype: object

In [14]:
articles_as_int = behaviors['history_int'].to_numpy()

In [15]:
behaviors['impressions_int'] = behaviors.impressions.str.split(' ')

In [16]:
behaviors['impressions_int_1'] = behaviors['impressions_int'].apply(lambda x: [article_idx[i[:-2]] for i in x if i[-1] == '1'])

In [17]:
impressions_1 = behaviors['impressions_int_1'].to_numpy()

In [18]:
behaviors['impressions_int_0'] = behaviors['impressions_int'].apply(lambda x: [article_idx[i[:-2]] for i in x if i[-1] == '0'])

In [19]:
impressions_0 = behaviors['impressions_int_0'].to_numpy()

In [20]:
complete_list_1s = []
for i in range(len(articles_as_int)):
    for art_int in articles_as_int[i]:
        complete_list_1s.append(art_int)
    complete_list_1s.append(impressions_1[i][0])
    


In [21]:
len(complete_list_1s)/6

136120.0

In [22]:
target_1s = [1]*136120

In [23]:
complete_list_0s = []
for i in range(len(articles_as_int)):
    for art_int in articles_as_int[i]:
        complete_list_0s.append(art_int)
    complete_list_0s.append(impressions_0[i][0])
    

In [24]:
len(complete_list_0s) /6

136120.0

In [25]:
len(complete_list_1s)/4 *3

612540.0

In [26]:
complete_list_1s_train = complete_list_1s[:612540]
complete_list_1s_test = complete_list_1s[612540:]

complete_list_0s_train = complete_list_0s[:612540]
complete_list_0s_test = complete_list_0s[612540:]

In [27]:
target_1s_train = [1]*(int(len(complete_list_1s_train)/6))
target_1s_test = [1]*(int(len(complete_list_1s_test)/6))

target_0s_train = [0]*(int(len(complete_list_0s_train)/6))
target_0s_test = [0]*(int(len(complete_list_0s_test)/6))

In [28]:
train = complete_list_1s_train + complete_list_0s_train
test = complete_list_1s_test + complete_list_0s_test

targets_train = target_1s_train + target_0s_train
targets_test = target_1s_test + target_0s_test

In [31]:
train = np.array(train)
test = np.array(test)
targets_train = np.array(targets_train)
targets_test = np.array(targets_test)

In [32]:
train = train.reshape(int(len(train) / 6), 6,1)
test = test.reshape(int(len(test) / 6), 6,1)

In [34]:
train[:3]

array([[[19662],
        [ 4343],
        [29194],
        [46913],
        [26811],
        [20693]],

       [[33409],
        [ 8855],
        [15451],
        [10733],
        [25150],
        [28117]],

       [[30923],
        [28544],
        [ 7090],
        [17478],
        [  680],
        [49238]]])

In [37]:
train.shape[0] == targets_train.shape[0]

True

In [66]:
test[0], behaviors.history_int[612540/6]

(array([[  234],
        [ 1605],
        [ 1261],
        [30114],
        [43860],
        [46265]]),
 [234, 1605, 1261, 30114, 43860])

In [68]:
behav = behaviors.to_numpy(copy=True)

In [72]:
612540/6

102090.0

In [77]:
unique_articels_int = [article_idx[art] for art in unique_articels]

In [89]:
all_negatives=[]
for session in behav[int(612540/6):]:
    negative_articles = set(unique_articels_int) - set([article_idx[art] for art in session[3]]) 
    negative_articles = list(negative_articles)
    
    negatives_to_rank =[]
    for t in range(99):
        j = np.random.randint(len(negative_articles))
        negatives_to_rank.append(negative_articles[j])
    all_negatives.append(negatives_to_rank)    

In [90]:
trajectories_to_score = test[:int(len(test)/2)]

In [91]:
len(all_negatives) == len(trajectories_to_score)

True

In [ ]:
test_negatives = []
for i, trajectory in enumerate(trajectories_to_score):
    negatives=[]
    for t in range(99):
        hist = trajectory[:5]
        hist = list(hist)
        print(hist)
        break
        negative = hist.append(all_negatives[i][t])
        negatives.append(negative)
    test_negatives.append(negatives)
    

In [ ]:
trajectories_to_score = test[:int(len(test)/2)]

In [54]:
trajectories_to_score.shape, test.shape[0]/2

((34030, 6, 1), 34030.0)

In [37]:
pos_trajectories, neg_trajector = []
for traj in trajectories_to_score:
    for art_id in traj:
    

[[55300]
 [57904]
 [66102]
 [ 5276]
 [30240]
 [32516]]


In [38]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=num_articles, output_dim=64))
model.add(layers.LSTM(128))
model.add(layers.Dense(10))
model.add(layers.Dense(1,activation='sigmoid'))

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          3189696   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 3,289,813
Trainable params: 3,289,813
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['AUC'])

In [42]:
history = model.fit(train, targets_train, epochs=1, validation_split=0.2, verbose=1)

5105/5105 [==============================] - 176s 35ms/step - loss: 0.6257 - auc: 0.6735 - val_loss: 0.9301 - val_auc: 0.0000e+00


In [46]:
test_input = test[0].reshape((1,6,1))

model.predict(test_input)

array([[0.97584033]], dtype=float32)

In [55]:
test_input_2 = trajectories_to_score.reshape((len(trajectories_to_score),6,1))
x = model.predict(test_input_2)

In [56]:
x.mean()

0.62638414

In [58]:
test_input_3 = test_negatives.reshape((len(test_negatives),6,1))
y = model.predict(test_input_3)

In [59]:
y.mean()

0.39724666